# 方法3: sportypy MLBField でフィールドを描く

**正確なMLB規格のフィールド** - sportypy パッケージを使用

| メリット | デメリット |
|----------|------------|
| 正確なMLB規格の寸法 | 追加パッケージが必要 |
| プロフェッショナルな見た目 | 座標変換は自分で行う |
| 他スポーツ対応（NHL, NBA, NFL等） | |

---

In [ ]:
!pip install pybaseball duckdb sportypy -q

In [ ]:
from pybaseball import statcast
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# sportypy
from sportypy.surfaces.baseball import MLBField

## 1. データ取得

In [ ]:
# ====== 設定 ======
BATTER_ID = 660271      # 大谷翔平 MLBAM ID
SEASON_YEAR = 2025
GAME_TYPE = "R"         # "R"=レギュラーシーズン, "P"=ポストシーズン, None=全試合
# ==================

# Statcastデータ取得
df_raw = statcast(start_dt=f'{SEASON_YEAR}-03-01', end_dt=f'{SEASON_YEAR}-12-31')
print(f"Total records (raw): {len(df_raw):,}")

# game_typeでフィルタ
con = duckdb.connect()
if GAME_TYPE:
    df = con.execute(f"""
        SELECT * FROM df_raw WHERE game_type = '{GAME_TYPE}'
    """).df()
    print(f"Filtered records (game_type='{GAME_TYPE}'): {len(df):,}")
else:
    df = df_raw.copy()
    print(f"Using all game types: {len(df):,}")

In [ ]:
# 大谷のデータを抽出
df_hits = con.execute("""
    SELECT * FROM df
    WHERE batter = 660271
      AND events IN ('home_run', 'double', 'triple', 'single')
      AND hc_x IS NOT NULL AND hc_y IS NOT NULL
""").df()

df_outs = con.execute("""
    SELECT * FROM df
    WHERE batter = 660271
      AND events NOT IN ('home_run', 'double', 'triple', 'single')
      AND hc_x IS NOT NULL AND hc_y IS NOT NULL
""").df()

df_hr = con.execute("""
    SELECT * FROM df
    WHERE batter = 660271 AND events = 'home_run'
      AND hc_x IS NOT NULL AND hc_y IS NOT NULL
""").df()

print(f"Hits: {len(df_hits)}, Outs: {len(df_outs)}, HR: {len(df_hr)}")

## 2. 座標変換

sportypy用の座標変換（方法2と同じ式）

In [ ]:
def transform_for_sportypy(df):
    """Statcast座標をsportypy用に変換
    
    sportypy MLBFieldは:
    - ホームプレートが原点
    - Y軸が外野方向
    """
    df = df.copy()
    df['x'] = 2.5 * (df['hc_x'] - 125.42)
    df['y'] = 2.5 * (198.27 - df['hc_y'])
    return df

df_hits_t = transform_for_sportypy(df_hits)
df_outs_t = transform_for_sportypy(df_outs)
df_hr_t = transform_for_sportypy(df_hr)

---
## 3. MLBField の基本表示

```python
mlb_field = MLBField()
fig, ax = mlb_field.draw()
```

これだけで正確なMLB規格のフィールドが描画される

In [ ]:
mlb_field = MLBField()
fig, ax = mlb_field.draw()
ax.set_title('MLB Field (sportypy)', fontsize=14)
plt.show()

---
## 4. 打球分布をプロット

In [ ]:
mlb_field = MLBField()
fig, ax = mlb_field.draw()

ax.scatter(df_outs_t['x'], df_outs_t['y'], c='blue', alpha=0.4, s=25, label='Outs')
ax.scatter(df_hits_t['x'], df_hits_t['y'], c='red', alpha=0.7, s=50, label='Hits')

ax.set_title('Ohtani 2025 - Batted Ball Distribution (sportypy)', fontsize=14)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## 5. ホームランのみ

In [ ]:
mlb_field = MLBField()
fig, ax = mlb_field.draw()

ax.scatter(df_hr_t['x'], df_hr_t['y'], c='red', alpha=0.8, s=100, 
           marker='*', label=f'Home Runs ({len(df_hr_t)})')

ax.set_title('Ohtani 2025 - Home Run Distribution (sportypy)', fontsize=14)
ax.legend(loc='upper right', fontsize=12)

plt.tight_layout()
plt.show()

---
## 6. イベント別カラー表示

In [ ]:
colors = {
    'single': 'green',
    'double': 'blue', 
    'triple': 'orange',
    'home_run': 'red'
}

mlb_field = MLBField()
fig, ax = mlb_field.draw()

for event, color in colors.items():
    subset = df_hits_t[df_hits_t['events'] == event]
    ax.scatter(subset['x'], subset['y'], c=color, alpha=0.7, 
               s=60, label=f"{event} ({len(subset)})")

ax.set_title('Ohtani 2025 - Hits by Type (sportypy)', fontsize=14)
ax.legend(loc='upper right', fontsize=10)

plt.tight_layout()
plt.show()

---
## 7. ヒートマップ + sportypy

**既存のaxesにsportypy描画 → ヒートマップ重ねる**

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))

# Outs
mlb_field1 = MLBField()
mlb_field1.draw(ax=axs[0])
sns.kdeplot(data=df_outs_t, x='x', y='y', ax=axs[0], 
            cmap='Blues', fill=True, alpha=0.5, levels=8)
axs[0].set_title('Ohtani 2025 - Outs Heatmap', fontsize=14)

# Hits
mlb_field2 = MLBField()
mlb_field2.draw(ax=axs[1])
sns.kdeplot(data=df_hits_t, x='x', y='y', ax=axs[1], 
            cmap='Reds', fill=True, alpha=0.5, levels=8)
axs[1].set_title('Ohtani 2025 - Hits Heatmap', fontsize=14)

plt.tight_layout()
plt.show()

## 8. Outs vs Hits 比較（横並び）

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 9))

# Outs
MLBField().draw(ax=axs[0])
axs[0].scatter(df_outs_t['x'], df_outs_t['y'], c='blue', alpha=0.5, s=40)
axs[0].set_title(f'Ohtani 2025 - Outs ({len(df_outs_t)})', fontsize=14)

# Hits
MLBField().draw(ax=axs[1])
axs[1].scatter(df_hits_t['x'], df_hits_t['y'], c='red', alpha=0.7, s=50)
axs[1].set_title(f'Ohtani 2025 - Hits ({len(df_hits_t)})', fontsize=14)

plt.suptitle('Shohei Ohtani 2025 - Batted Ball Comparison', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

---
## まとめ

**sportypy MLBField のポイント:**

1. **最も見た目がプロフェッショナル** - 正確なMLB規格

2. **基本的な使い方**
```python
from sportypy.surfaces.baseball import MLBField

mlb_field = MLBField()
fig, ax = mlb_field.draw()  # 新規figure作成
# または
mlb_field.draw(ax=existing_ax)  # 既存axesに描画
```

3. **座標変換は方法2と同じ**
```python
x = 2.5 * (hc_x - 125.42)
y = 2.5 * (198.27 - hc_y)
```

4. **ヒートマップとの組み合わせ**
```python
MLBField().draw(ax=ax)
sns.kdeplot(data=df, x='x', y='y', ax=ax, cmap='Reds', fill=True)
```